In [1]:
from pathlib import Path
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Optional: progress bar
try:
    from tqdm.auto import tqdm
except ImportError:
    tqdm = lambda x, **kwargs: x  # fallback

# Base paths (relative to notebooks/)
PROJECT_ROOT = Path("..").resolve()
DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"

FEATURES_PATH = DATA_PROCESSED / "combined_features.csv"
MPD_DIR = DATA_RAW / "mpd"

print("Project root:", PROJECT_ROOT)
print("Features CSV:", FEATURES_PATH)
print("MPD dir:", MPD_DIR)


Project root: C:\Users\Kiera\Music_Recommender
Features CSV: C:\Users\Kiera\Music_Recommender\data\processed\combined_features.csv
MPD dir: C:\Users\Kiera\Music_Recommender\data\raw\mpd


c:\Users\Kiera\Music_Recommender\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
features_df = pd.read_csv(FEATURES_PATH)

print("combined_features.csv shape:", features_df.shape)
print("Columns:", list(features_df.columns))

features_df.head()


combined_features.csv shape: (4589288, 17)
Columns: ['track_id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'year', 'explicit', 'time_signature', 'popularity']


,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,year,explicit,time_signature,popularity
0,0000QuApNltQzqS5ROXcQ7,0.629,0.199,4,-15.258,1,0.0610,0.9250,0.000000,0.1100,0.587,66.469,141813.0,2012.0,NaN,4.0,NaN
1,0000j9zugybaOxAeENtFRU,0.680,0.378,5,-7.379,1,0.0685,0.8850,0.000013,0.1200,0.963,116.877,112107.0,NaN,NaN,4.0,NaN
2,0000korRHja9p9XaR5UA5m,0.708,0.714,6,-7.579,1,0.0362,0.3970,0.000047,0.3540,0.756,120.538,235666.0,1995.0,NaN,4.0,NaN
3,0000uJA4xCdxThagdLkkLR,0.458,0.591,5,-5.621,1,0.0326,0.5680,0.000015,0.2860,0.654,184.913,161187.0,NaN,NaN,3.0,NaN
4,00018DRtkxXS7SmAG4ciN6,0.456,0.324,8,-11.521,1,0.0342,0.0242,0.596000,0.0868,0.522,184.970,238827.0,NaN,NaN,3.0,NaN


In [3]:
def extract_track_id_from_uri(uri: str) -> str | None:
    """
    Convert 'spotify:track:0UaMYEvWZi0ZqiDOoHU3YI' -> '0UaMYEvWZi0ZqiDOoHU3YI'.
    """
    if not isinstance(uri, str):
        return None
    parts = uri.split(":")
    return parts[-1] if len(parts) >= 3 else None

mpd_files = sorted(MPD_DIR.rglob("mpd.slice.*.json"))
print(f"Found {len(mpd_files)} MPD slice files")

mpd_track_ids = set()
total_playlists = 0
total_track_entries = 0

for path in tqdm(mpd_files, desc="Scanning MPD slices"):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    playlists = data.get("playlists", [])
    total_playlists += len(playlists)

    for pl in playlists:
        tracks = pl.get("tracks", [])
        total_track_entries += len(tracks)
        for t in tracks:
            tid = extract_track_id_from_uri(t.get("track_uri"))
            if tid:
                mpd_track_ids.add(tid)

print("\n=== MPD Stats ===")
print(f"Total playlists:                       {total_playlists:,}")
print(f"Total track entries (with duplicates): {total_track_entries:,}")
print(f"Unique track IDs in MPD:               {len(mpd_track_ids):,}")


Found 100 MPD slice files


Scanning MPD slices: 100%|██████████| 100/100 [00:23<00:00,  4.19it/s]


=== MPD Stats ===
Total playlists:                       100,000
Total track entries (with duplicates): 6,685,101
Unique track IDs in MPD:               679,889


In [4]:
features_track_ids = set(features_df["track_id"].astype(str))
intersection_ids = features_track_ids & mpd_track_ids

print("=== Overlap Summary ===")
print(f"Unique track IDs in combined_features: {len(features_track_ids):,}")
print(f"Unique track IDs in MPD:               {len(mpd_track_ids):,}")
print(f"Overlap (tracks present in BOTH):      {len(intersection_ids):,}")
print(f"  - % of MPD covered by features:      {len(intersection_ids) / len(mpd_track_ids) * 100:.2f}%")
print(f"  - % of features in MPD:              {len(intersection_ids) / len(features_track_ids) * 100:.2f}%")

# Filter features_df down to only overlapping track_ids
merged_df = features_df[features_df["track_id"].astype(str).isin(intersection_ids)].copy()

print("\nmerged_df shape (features only for tracks that appear in MPD):", merged_df.shape)
merged_df.head()


=== Overlap Summary ===
Unique track IDs in combined_features: 4,589,288
Unique track IDs in MPD:               679,889
Overlap (tracks present in BOTH):      514,672
  - % of MPD covered by features:      75.70%
  - % of features in MPD:              11.21%

merged_df shape (features only for tracks that appear in MPD): (514672, 17)


,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,year,explicit,time_signature,popularity
3,0000uJA4xCdxThagdLkkLR,0.458,0.5910,5,-5.621,1,0.0326,0.5680,0.000015,0.286,0.654,184.913,161187.0,NaN,NaN,3.0,NaN
8,00039MgrmLoIzSpuYKurn9,0.742,0.7530,1,-5.632,1,0.0364,0.0178,0.000000,0.133,0.263,132.064,222727.0,NaN,NaN,4.0,NaN
14,0005w1bMJ7QAMl6DY98oxa,0.253,0.1970,6,-17.695,1,0.0461,0.8830,0.005620,0.152,0.377,61.556,111573.0,NaN,NaN,4.0,NaN
56,000JBgYWfJQqdFaRqu2n3f,0.496,0.0481,7,-26.668,0,0.0520,0.9790,0.767000,0.110,0.125,79.477,227627.0,NaN,NaN,4.0,NaN
57,000JCyEkMFumqCZQJAORiQ,0.795,0.8740,0,-4.523,1,0.2100,0.0646,0.000000,0.341,0.483,132.966,207125.0,NaN,NaN,4.0,NaN


In [5]:
cols_of_interest = ["explicit", "popularity", "year", "time_signature", "duration_ms"]

missing_summary_rows = []
n_rows = len(merged_df)

for col in cols_of_interest:
    if col not in merged_df.columns:
        continue

    missing_count = merged_df[col].isna().sum()
    missing_pct = missing_count / n_rows * 100

    # For numeric cols, we can also count zeros
    if pd.api.types.is_numeric_dtype(merged_df[col]):
        zero_count = (merged_df[col] == 0).sum()
        zero_pct = zero_count / n_rows * 100
    else:
        zero_count = np.nan
        zero_pct = np.nan

    missing_summary_rows.append({
        "feature": col,
        "dtype": str(merged_df[col].dtype),
        "n_rows": n_rows,
        "missing_count": missing_count,
        "missing_pct": round(missing_pct, 3),
        "zero_count": zero_count,
        "zero_pct": round(zero_pct, 3) if pd.notna(zero_pct) else np.nan,
    })

missing_overlap_df = pd.DataFrame(missing_summary_rows)
print("=== Missing / Zero Summary for Key Columns (in MPD ∩ features subset) ===")
missing_overlap_df


=== Missing / Zero Summary for Key Columns (in MPD ∩ features subset) ===


,feature,dtype,n_rows,missing_count,missing_pct,zero_count,zero_pct
0,explicit,object,514672,402942,78.291,NaN,NaN
1,popularity,float64,514672,364657,70.852,180.0,0.035
2,year,float64,514672,353033,68.594,0.0,0.000
3,time_signature,float64,514672,1615,0.314,428.0,0.083
4,duration_ms,float64,514672,6016,1.169,0.0,0.000


In [6]:
numeric_cols = [
    c for c in merged_df.columns
    if c != "track_id" and pd.api.types.is_numeric_dtype(merged_df[c])
]

rows = []
n_rows = len(merged_df)

for col in numeric_cols:
    missing_count = merged_df[col].isna().sum()
    missing_pct = missing_count / n_rows * 100

    zero_count = (merged_df[col] == 0).sum()
    zero_pct = zero_count / n_rows * 100

    rows.append({
        "feature": col,
        "dtype": str(merged_df[col].dtype),
        "n_rows": n_rows,
        "missing_count": missing_count,
        "missing_pct": round(missing_pct, 3),
        "zero_count": zero_count,
        "zero_pct": round(zero_pct, 3),
        "valid_nonzero_count": n_rows - missing_count - zero_count,
    })

coverage_overlap_df = (
    pd.DataFrame(rows)
    .sort_values(by=["missing_pct", "zero_pct"], ascending=[False, False])
    .reset_index(drop=True)
)

print("=== Missing / Zero Summary for ALL numeric features (only overlap rows) ===")
coverage_overlap_df


=== Missing / Zero Summary for ALL numeric features (only overlap rows) ===


,feature,dtype,n_rows,missing_count,missing_pct,zero_count,zero_pct,valid_nonzero_count
0,popularity,float64,514672,364657,70.852,180,0.035,149835
1,year,float64,514672,353033,68.594,0,0.000,161639
2,duration_ms,float64,514672,6016,1.169,0,0.000,508656
3,time_signature,float64,514672,1615,0.314,428,0.083,512629
4,mode,int64,514672,0,0.000,175661,34.131,339011
5,instrumentalness,float64,514672,0,0.000,155617,30.236,359055
6,key,int64,514672,0,0.000,61512,11.952,453160
7,valence,float64,514672,0,0.000,521,0.101,514151
8,danceability,float64,514672,0,0.000,417,0.081,514255
9,speechiness,float64,514672,0,0.000,417,0.081,514255


In [7]:
def missing_stats(df, cols):
    rows = []
    n = len(df)
    for col in cols:
        if col not in df.columns:
            continue
        mc = df[col].isna().sum()
        mp = mc / n * 100
        rows.append({
            "feature": col,
            "n_rows": n,
            "missing_count": mc,
            "missing_pct": round(mp, 3),
        })
    return pd.DataFrame(rows)

original_missing = missing_stats(features_df, cols_of_interest)
overlap_missing = missing_stats(merged_df, cols_of_interest)

original_missing["scope"] = "all_features"
overlap_missing["scope"] = "overlap_only"

compare_missing = pd.concat([original_missing, overlap_missing], ignore_index=True)
compare_missing = compare_missing.pivot(index="feature", columns="scope", values="missing_pct")

print("=== Missing % in original vs overlap-only ===")
compare_missing


=== Missing % in original vs overlap-only ===


scope,all_features,overlap_only
feature,,
duration_ms,0.131,1.169
explicit,60.792,78.291
popularity,59.542,70.852
time_signature,0.478,0.314
year,46.512,68.594
